# Desafios idwall


## Resolução da Parte 1 do Desafio 2 - Crawlers

Web Scraping do Reddit, escrito em Python, utilizando as bibliotecas _request_ e _BeautifulSoup_.

#### Desafio:
Encontrar e listar as _threads_ com 5000 pontos ou mais no Reddit naquele momento.


### Resolução:

É possível realizar o scrapping via PRAW, um wrapper para o API do Reddit, o qual permite realizar scrapings dos subreddits, criar um bot, entre outras funcionalidades.

Seguem dois sites que ensinam como realizar scrapping por esse método:

https://towardsdatascience.com/scraping-reddit-data-1c0af3040768

http://www.storybench.org/how-to-scrape-reddit-with-python/


Mas aqui, com o intuito de demonstrar habilidades mais gerais, vamos realizar o scrapping utilizando os pacotes 'request' e 'BeautifulSoup'.

Para uma breve introdução sobre web scraping e aplicação destes pacotes ver:

https://www.scrapehero.com/a-beginners-guide-to-web-scraping-part-1-the-basics/

https://www.youtube.com/watch?v=ng2o98k983k&t=1428s


Dito isto, vamos começar pelo simples e buscar as _top threads_ dentro do subreddit 'r/AskReddit': https://www.reddit.com/r/AskReddit/top/?t=day

"Subreddits são como fóruns dentro do Reddit e as postagens são chamadas threads.

Para quem gosta de gatos, há o subreddit '/r/cats' com threads contendo fotos de gatos fofinhos. Para threads sobre o Brasil, vale a pena visitar '/r/brazil' ou ainda '/r/worldnews'. Um dos maiores subreddits é o '/r/AskReddit'."

### Abrindo a url e salvando o arquivo em html:

In [1]:
# Primeiramente, importam-se as bibliotecas necessárias para o scrapping:
import requests
from bs4 import BeautifulSoup

# E cria-se uma função para salvar e outra para
# abrir a página html, a fim de minimizar danos ao servidor:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

def open_html(path):
    with open(path, 'rb') as f:
        return f.read()
    
# Eu deixei essas funções salvas num arquivo chamado
# save_open_html.py para consultas futuras, se necessário

#### ATENÇÃO!!!

Recomenda-se não rodar o código da célula abaixo, sendo utilizado assim apenas para simples conferência.

Como o reddit possui um sistema automatizado que impede mais que um request a cada dois segundos é possível que o código abaixo gere um "erro", de forma a não ser capaz de obter o código html do site. Eu tentei procurar entender o porquê, mas não obtive uma resposta.

Mas caso queira rodar o código, é necessário tentar algumas vezes, caso não consiga de primeira, até conseguir obter o html.

In [2]:
# Para abrir localmente e trabalhar com o arquivo e não com vários requests:
html = open_html('askreddit_top_day')

soup = BeautifulSoup(html, 'lxml')

print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <script>
   var __SUPPORTS_TIMING_API = typeof performance === 'object' && !!performance.mark && !! performance.measure && !!performance.getEntriesByType;
          function __perfMark(name) { __SUPPORTS_TIMING_API && performance.mark(name); };
          var __firstLoaded = false;
          function __markFirstPostVisible() {
            if (__firstLoaded) { return; }
            __firstLoaded = true;
            __perfMark("first_post_title_image_loaded");
          }
  </script>
  <script>
   __perfMark('head_tag_start');
  </script>
  <title>
   Ask Reddit...
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="origin-when-cross-origin" name="referrer"/>
  <style>
   /* http://meyerweb.com/eric/tools/css/reset/
    v2.0 | 20110126
    License: none (public domain)
  */

  html, body, div, span, applet, object, iframe,
  h1, h2, h3, h4, h5, h6, p, blockquote, pre,
  a, 

### Realizando o parsing no código reddit:

Primeiramente vamos analisar se o primeiro top thread é o desejado:

In [3]:
## html do bloco completo contendo todos os comentários
all_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

## html da tag e classe do número de pontos de certa thread:
# <div class="_1rZYMD_4xY3gRcSS3p8ODO" style="color:#1A1A1B">22.8k</div>
pontos_thread = all_threads.find('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
pontos_primeira_thread = pontos_thread.text

## html da tag e classe do texto de uma certa thread
# <h3 class="_eYtD2XCVieq6emjKBH3m">Teachers of Reddit, what was the most obvious "teacher crush" someone had on you?</h3>
thread = all_threads.find('h3', class_="_eYtD2XCVieq6emjKBH3m")
texto_thread = thread.text

## html do link do texto
# <a data-click-id="body" class="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE" href="/r/AskReddit/comments/cyqtk2/teachers_of_reddit_what_was_the_most_obvious/"><div class="_2SdHzo12ISmrC8H86TgSCp _3wqmjmv3tb_k-PROt7qFZe " style="--posttitletextcolor:#444e59" theme="[object Object]"><h3 class="_eYtD2XCVieq6emjKBH3m">Teachers of Reddit, what was the most obvious "teacher crush" someone had on you?</h3></div></a>
link = all_threads.find('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")
referencia = link['href']
texto_link = f'https://www.reddit.com{referencia}'

In [4]:
# Resultado dos pontos e texto do primeiro top thread do dia:
print(pontos_primeira_thread)
print(texto_thread)
print(texto_link)

28.8k
Everyone has a scar on their body from something dumb, they did as a child. What's your story?
https://www.reddit.com/r/AskReddit/comments/cz2apy/everyone_has_a_scar_on_their_body_from_something/


Tudo certo até aqui, então vamos seguir com os próximos passos.

### Realizando um loop sobre todas as top threads desejadas:

In [5]:
table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

extracted_points = []
for points in all_points_thread:
    point = points.text
    extracted_points.append(point)

extracted_texts = []
for threads in all_texts:
    thread = threads.text
    extracted_texts.append(thread)
    
extracted_links = []
for links in all_links:
    referencia = links['href']
    if referencia.startswith('http'):
        extracted_links.append(referencia)
    else:
        texto_link = f'https://www.reddit.com{referencia}'
        extracted_links.append(texto_link)
        
print(len(extracted_points))
print(len(extracted_texts))

16
8


Nesse código, é possivel ver que a lista contendo as pontuações está duplicada. Provavelmente isto ocorre por haverem classes repitidas no código html. Para consertar isto, basta criarmos uma nova lista contendo os valores únicos:

In [6]:
print(extracted_points)
unique_extracted_points = []
for i in range(0, len(extracted_points), 2):
    unique_extracted_points.append(extracted_points[i])
    
print(unique_extracted_points)

['28.8k', '28.8k', '3.3k', '3.3k', '2.7k', '2.7k', '1.2k', '1.2k', '1.5k', '1.5k', '812', '812', '936', '936', '563', '563']
['28.8k', '3.3k', '2.7k', '1.2k', '1.5k', '812', '936', '563']


In [7]:
top_threads = []
for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
    if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
        likes = p[0:-1]
        likes = int(float(likes)*1000)
        if likes >= 5000:
            subreddit = l.split('/')[4]
            s = f'/r/{subreddit}'
            record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
            top_threads.append(record)
            
print(top_threads)

[{'pontuacao': '28.8k', 'subreddit': '/r/AskReddit', 'titulo thread': "Everyone has a scar on their body from something dumb, they did as a child. What's your story?", 'link para os comentarios': 'https://www.reddit.com/r/AskReddit/comments/cz2apy/everyone_has_a_scar_on_their_body_from_something/'}]


In [8]:
# Opcionalmente podemos salvar o resultando em um arquivo json ou csv para uso futuro

# import json

# with open('data.json', 'w') as outfile:
#     json.dump(top_threads, outfile, indent=4)
    
import csv
csv_file = open('data.csv', 'w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['pontuacao', 'subreddit', 'thread', 'link'])

# inserir csv_writer.writerow([p, s , t , l]) no loop das "top_threads = []"

33

### Realizando o scrapping na pagina principal

Agora vamos ir para uma página mais geral do site www.reddit.com, e realizar um novo scrape (Novamente, a célula abaixo foi convertida em Raw NBConvert, para evitar rodá-la acidentalmente):

In [9]:
# Código restante:

html = open_html('reddit_top_today')

soup = BeautifulSoup(html, 'lxml')

table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

extracted_points = []
for points in all_points_thread:
    point = points.text
    extracted_points.append(point)

extracted_texts = []
for threads in all_texts:
    thread = threads.text
    extracted_texts.append(thread)
    
extracted_links = []
for links in all_links:
    referencia = links['href']
    if referencia.startswith('http'):
        extracted_links.append(referencia)
    else:
        texto_link = f'https://www.reddit.com{referencia}'
        extracted_links.append(texto_link)

unique_extracted_points = []
for i in range(0, len(extracted_points), 2):
    unique_extracted_points.append(extracted_points[i])

top_threads = []
for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
    if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
        likes = p[0:-1]
        likes = int(float(likes)*1000)
        if likes >= 5000:
            subreddit = l.split('/')[4]
            s = f'/r/{subreddit}'
            record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
            top_threads.append(record)
            csv_writer.writerow([p, s , t , l])

csv_file.close()
for i in range(len(top_threads)):
    print(top_threads[i])
    print()

{'pontuacao': '121k', 'subreddit': '/r/aww', 'titulo thread': 'Scared cat gets saved by two French guys', 'link para os comentarios': 'https://www.reddit.com/r/aww/comments/cyv97r/scared_cat_gets_saved_by_two_french_guys/'}

{'pontuacao': '111k', 'subreddit': '/r/memes', 'titulo thread': 'The Area 51 raid is still happening right?', 'link para os comentarios': 'https://www.reddit.com/r/memes/comments/cz2i20/the_area_51_raid_is_still_happening_right/'}

{'pontuacao': '106k', 'subreddit': '/r/pics', 'titulo thread': 'In 1964, Ringo Starr snapped a photo of some high school students who skipped class to see the Beatles during their first trip to the US. The group had no idea the photo existed until Ringo published his book of photos. Nearly 50 years later, the group reunited and recreated the photo.', 'link para os comentarios': 'https://www.reddit.com/r/pics/comments/cyx1os/in_1964_ringo_starr_snapped_a_photo_of_some_high/'}

{'pontuacao': '103k', 'subreddit': '/r/aww', 'titulo thread': 

### Realizando o scrapping em uma lista de subreddits

Agora que entendemos como realizar o scrappping em um único link, o próximo passo seria realizar o scraping a partir de uma lista de subreddits separados por ponto-e-vírgula, e.g., "programming;dogs;brazil".

In [10]:
# input: askreddit;worldnews;cats
subreddits = str(input('Quais subreddits gostaria de acompanhar hoje? \n'))
subreddits_separated = subreddits.split(';')

print('\n', subreddits_separated)

Quais subreddits gostaria de acompanhar hoje? 
askreddit;worldnews;cats

 ['askreddit', 'worldnews', 'cats']


In [11]:
# A partir da lista criada anteriormente, cria-se outra lista com as urls para download do código html
urls_subreddits = []
for i in range(len(subreddits_separated)):
    url_link = 'https://www.reddit.com/r/{}/top/?t=day'.format(subreddits_separated[i])
    urls_subreddits.append(url_link)

print(urls_subreddits)

['https://www.reddit.com/r/askreddit/top/?t=day', 'https://www.reddit.com/r/worldnews/top/?t=day', 'https://www.reddit.com/r/cats/top/?t=day']


Indo para a pasta principal, é possível ver que os arquivos foram salvos. Mas tem casos em que rodando o código acima, por conta da proteção que o site reddit possui, os htmls salvos não correspondem aos desejados. Nesses casos é necessária uma intervenção humana no looping para ser possível obter os htmls desejados, tendo que baixar um por vez.

A partir da lista de htmls é possível realizar o mesmo scrapping anterior utilizando um loop, como mostrado abaixo: 

In [12]:
for i in subreddits_separated:
    
    html = open_html(f'reddit_{i}')

    soup = BeautifulSoup(html, 'lxml')

    table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

    all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
    all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
    all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

    # Nesses dois casos (extracted_points e extracted_texts), é possível criarmos uma função, 
    # mas aqui como achei que o código não possui uma recorrência alta desses loops
    # preferi deixá-los explícitos
    extracted_points = []
    for points in all_points_thread:
        point = points.text
        extracted_points.append(point)

    extracted_texts = []
    for threads in all_texts:
        thread = threads.text
        extracted_texts.append(thread) 

    extracted_links = []
    for links in all_links:
        referencia = links['href']
        if referencia.startswith('http'):
            extracted_links.append(referencia)
        else:
            texto_link = f'https://www.reddit.com{referencia}'
            extracted_links.append(texto_link)
    
    unique_extracted_points = []
    for j in range(0, len(extracted_points), 2):
        unique_extracted_points.append(extracted_points[j])

    top_threads = []
    for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
        if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
            likes = p[0:-1]
            likes = int(float(likes)*1000)
            if likes >= 5000:
                subreddit = l.split('/')[4]
                s = f'/r/{subreddit}'
                record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
                top_threads.append(record)

    print(f'Top threads de /r/{i}: ', top_threads)
    print()

Top threads de /r/askreddit:  [{'pontuacao': '33.4k', 'subreddit': '/r/AskReddit', 'titulo thread': "What do people THINK is a scam, but they actually just don't understand it?", 'link para os comentarios': 'https://www.reddit.com/r/AskReddit/comments/d0dlvo/what_do_people_think_is_a_scam_but_they_actually/'}, {'pontuacao': '30.9k', 'subreddit': '/r/AskReddit', 'titulo thread': "What's a negative fact about marijuana we should know?", 'link para os comentarios': 'https://www.reddit.com/r/AskReddit/comments/d0aoi5/whats_a_negative_fact_about_marijuana_we_should/'}, {'pontuacao': '40.2k', 'subreddit': '/r/AskReddit', 'titulo thread': "The 2010's decade will be over in 4 months. What do you think people will remember this decade for?", 'link para os comentarios': 'https://www.reddit.com/r/AskReddit/comments/d0jjc2/the_2010s_decade_will_be_over_in_4_months_what_do/'}, {'pontuacao': '24.5k', 'subreddit': '/r/AskReddit', 'titulo thread': "What's the worst case you've seen of someone trying t

Podemos também modificar o limite de pontos para um mínimo de 500 pontos por exemplo. Como mostrado abaixo:

No caso acima, juntamente com os threads, também mandei imprimir os valores dos likes para compararmos os resultados e tentar buscar por erros. Se rodarmos o código, aparentemente estará tudo certo.

Nos casos em que não temos threads com o valor de pontos desejados, nosso código não retornaria resultados para esses casos, retornando ao invés disso a resposta "Nao ha top threads com os requisitos [...]".

## Resolução "extra" - Função para 'reddit scrapping':
Por fim, podemos criar uma função que recebe os subreddits desejados e retorna as top threads:

In [26]:
from save_open_html import *

def top_threads(subreddits):
#     import requests
    from bs4 import BeautifulSoup
    
#     subreddits = str(input('Quais subreddits gostaria de acompanhar hoje? \n'))
    subreddits_separated = subreddits.split(';')
    
    urls_subreddits = []
    for i in range(len(subreddits_separated)):
        url_link = 'https://www.reddit.com/r/{}/top/?t=day'.format(subreddits_separated[i]) 
        urls_subreddits.append(url_link)
    
#     Aqui salvaríamos os htmls usando um for loop:
#
#     for i in range(0, len(urls_subreddits)):
#         url = urls_subreddits[i]
#         codigo_html = requests.get(url)
#         save_html(codigo_html.content, f'reddit_{subreddits_separated[i]}')

    for i in subreddits_separated:

        html = open_html(f'reddit_{i}')

        soup = BeautifulSoup(html, 'lxml')

        table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

        all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
        all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
        all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

        extracted_points = []
        for points in all_points_thread:
            point = points.text
            extracted_points.append(point)

        extracted_texts = []
        for threads in all_texts:
            thread = threads.text
            extracted_texts.append(thread)

        extracted_links = []
        for links in all_links:
            referencia = links['href']
            if referencia.startswith('http'):
                extracted_links.append(referencia)
            else:
                texto_link = f'https://www.reddit.com{referencia}'
                extracted_links.append(texto_link)

        unique_extracted_points = []
        for j in range(0, len(extracted_points), 2):
            unique_extracted_points.append(extracted_points[j])

        top_threads = []
        for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
            if len(p) > 1: # Note que a partir daqui, o código foi modificado com o fim de obter as threads com pontuação menor que 1k
                likes = p
                if p[-1] == 'k':
                    likes = p[0:-1]
                    likes = int(float(likes)*1000)
                else:
                    likes = int(likes)
                if likes >= 5000:
                    subreddit = l.split('/')[4]
                    s = f'/r/{subreddit}'
                    record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
                    top_threads.append(record)
        if top_threads == []:
            top_threads = "Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)"
        print()
        print(f'Top threads de /r/{i}: ', top_threads)

In [15]:
# inputs posíveis: programming;dogs;brazil;askreddit;worldnews;cats
top_threads('programming;cats;worldnews')


Top threads de /r/programming:  Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)

Top threads de /r/cats:  [{'pontuacao': '12.9k', 'subreddit': '/r/cats', 'titulo thread': 'Stray kitten after being de-flead, fed and bathed', 'link para os comentarios': 'https://www.reddit.com/r/cats/comments/d09fx6/stray_kitten_after_being_deflead_fed_and_bathed/'}, {'pontuacao': '11.7k', 'subreddit': '/r/cats', 'titulo thread': "Don't have many friends to share my cat with so here's how he sleeps", 'link para os comentarios': 'https://www.reddit.com/r/cats/comments/d0ewwv/im_so_lucky_to_have_her/'}, {'pontuacao': '9.1k', 'subreddit': '/r/cats', 'titulo thread': 'I’m so lucky to have her.', 'link para os comentarios': 'https://www.reddit.com/r/cats/comments/d0fovd/sweet_kitty_welcomes_returning_soldier/'}]

Top threads de /r/worldnews:  [{'pontuacao': '33.1k', 'subreddit': '/r/worldnews', 'titulo thread': 'Robert Mugabe dies aged 95', 'link para os comentarios': 'https://www.r

## Resolução - Parte 2: Bot para Telegram

### Criando um bot

A seguir, preferi colocar um código de bot (um pouco) mais completo com o intuito de reutilizar esse código futuramente (seja para revisar alguns conceitos, quanto para poder implementar os comandos após as hashtags - afinal, também me divirto trabalhando hehe)

A base do código foi retirada do 'Curso de Python na prática', presente no youtube, e gravado pelo usuário 11Wills11:

https://www.youtube.com/playlist?list=PLsMpSZTgkF5C_Kkc0XBtM3OVLBsjUkjzy

Esse bot é capaz de responder frases simples, tais como 'oi', 'tchau'. Também é possível ensinar novas frases utilizando o comando 'aprende'. Ele também é "customizado" para tratar diferentemente quem ele "conhece" e desconhecidos.

Quem ele não conhece, ele retorna "Muito Prazer (nome)!", seguido de uma pergunta sobre os threads. E quem ele conhece, retorna "oi (nome)! te adoro, sua maravilhosa!", seguido, também, de uma pergunta sobre os threads.

Após o código, existem algumas demonstrações.

Segue o bot com algumas modificações:

In [25]:
# Criando um bot básico:
# import json
# import sys
# import os
# import subprocess as sub
from save_open_html import *

class Chatbot():
    # Na inicialização '__init__(self,nome)' desta classe, a lista de conhecidos
    # e um histórico da conversa atual são inicializados junto do
    # nome do Chatbot, sendo "memorizado" para consultas futuras.
    def __init__(self, nome):
#         Esses comandos após as hashtags, são comandos para implementar um aprendizado de nomes
#
#         try:
#             memoria = open(nome+'.json', 'r')
#         except FileNotFoundError:
#             memoria = open(nome+'.json', 'w')
#             memoria.write('["Criadora", "Lais"], {"oi": "oi! qual o seu nome?", "tchau": "tchau!", "nenhum": "ok... sinto muito :("}')
#             memoria.close()
#             memoria = open(nome+'.json', 'r')
        self.nome = nome
        self.conhecidos = ['Lais', 'Renata', 'Mariana']
#         self.conhecidos, self.frases = json.load(memoria)
#         memoria.close()
        self.historico = [None]
        self.frases = {'oi': 'oi! qual o seu nome?', 'tchau': 'tchau!', 'nenhum': 'ok... sinto muito :('}
    
    def escuta(self, frase=None):
        if frase == None:
            frase = input('>:')
        frase = str(frase)
        if 'executa ' in frase:
            return frase
        frase = frase.lower()
        return frase
    
    def pensa(self, frase):
        if frase in self.frases:
            return self.frases[frase]
        
        if frase == 'aprende':
#             chave = input('Digite a frase: ')
#             resp = input('Digite a resposta: ')
#             self.frases[chave] = resp
#             return 'Aprendido'

#       No caso do telegram, temos que substituir os comandos acima para que o bot funcione
            return 'Digite a frase: '
        
#       Os comandos abaixo servem para responder frases que dependem do historico
        ultimaFrase = self.historico[-1]
        if ultimaFrase == 'oi! qual o seu nome?':
            nome = self.pega_nome(frase)
            resp = self.responde_nome(nome)
            return resp
        if ultimaFrase == 'Digite a frase: ':
            self.chave = frase
            return 'Digite a resposta: '
        if ultimaFrase == 'Digite a resposta: ':
            resp = frase
            self.frases[self.chave] = resp
#             gravaMemoria()
            return 'Aprendido'

        if '/nadaprafazer ' in frase:
            subreddit = frase.replace('/nadaprafazer ', '')
            try:
                return top_threads(subreddit)
            except:
                pass
        try:
            resp = srt(eval(frase))
            return resp
        except:
            pass

        return 'nao te entendi o.o'
    
    def pega_nome(self, nome):
        nome = nome.split(' ')
        if len(nome) == 1:
            nome = nome[0]
        else:
            nome = nome[-1]
        nome = nome.title()
        return nome
    
    def responde_nome(self, nome):
        if nome in self.conhecidos:
            frase = 'oi '
#             print(frase+nome+'! te adoro, sua maravilhosa!')
        else:
            frase = 'Muito prazer '
        return frase+nome+'! '
#             Estes códigos seriam para criar um arquivo contendo todas as pessoas que o bot já conversou, para ser implementado junto dos comandos comentados acima (em __init__)
#
#             self.conhecidos.append(nome)
#             memoria = open(self.nome+'.json', 'w')
#             json.dump(self.conhecidos, memoria)
#             memoria.close()
#       ou
#             self.gravaMemoria()
            
#         return 'O que gostaria gostaria de acompanhar hoje?' 
        
    def fala(self, frase):
#        Esses comandos abaixo após as hashtags seriam para implementar outra funcionalidade no bot: a de abrir documentos, programas e paginas de internet
#        No caso do linux, podemos escrever para o bot "executa chromium", por exemplo, para que ele abra o navegador.
#        Ou por exemplo "executa https://www.google.com.br", para abrir o site
#
#         if 'executa ' in frase:
#             plataforma = sys.plataform
#             comando = frase.replace('executa ', '')
#             if 'win' in plataforma:
#                 os.startfile(comando)
#             if 'linux' in plataforma:
#                 try:
#                     sub.Popen(comando)
#                 except FileNotFoundError:
#                     sub.Popen(['xdg-open', comando])

#         else:
        print(frase)
        self.historico.append(frase)
        
#         def gravaMemoria(self):
#             memoria = open(self.nome+'.json', 'w')
#             json.dump([self.conhecidos, self.frases], memoria)
#             memoria.close()
    
    

O bot funciona com a lógica: Escuta --> Pensa --> Fala.

Dentro da Classe 'Chatbot', temos essas funções que o caracterizam e mais outras duas (pega_nome, responde_nome) com o intuito de otimizar a leitura do código.

Para que o bot fique completo, precisamos de um loop que o mantenha "ativo":

In [59]:
#### APENAS EXEMPLO! EVITAR RODAR A CELULA ####
Bot = Chatbot('Red - O scrappista')

while True:
    frase = Bot.escuta()
    resp = Bot.pensa(frase)
    Bot.fala(resp)
    if resp == 'tchau!':
        break

>:oi
oi! qual o seu nome?
>:Lais, sua criadora
Muito prazer Criadora!
Quais threads gostaria de acompanhar hoje?
>:nenhum
ok... sinto muito :(
>:tchau
tchau!


In [17]:
# Assim, podemos criar uma função que executa o bot em questão:

def executa_bot(nome_do_bot):
    Bot = Chatbot(nome_do_bot)
    while True:
        frase = Bot.escuta()
        resp = Bot.pensa(frase)
        Bot.fala(resp)
        if resp == 'tchau!':
            break

In [11]:
### APENAS EXEMPLO! EVITAR RODAR A CELULA ###
executa_bot('Red - O scrappista')

>:oi
oi! qual o seu nome?
>:Mariana
oi Mariana! te adoro, sua maravilhosa!
Quais threads gostaria de acompanhar hoje?
>:nenhum
ok... sinto muito :(
>:tchau
tchau!


In [27]:
# Utilize este para "brincar", ou testar, o código,
# lembrando que o bot ainda não sabe retornar as top threads :)

executa_bot('Red - O scrappista')

>:oi
oi! qual o seu nome?
>:Laís
Muito prazer Laís! 
>:/NadaPraFazer cats

Top threads de /r/cats:  [{'pontuacao': '12.9k', 'subreddit': '/r/cats', 'titulo thread': 'Stray kitten after being de-flead, fed and bathed', 'link para os comentarios': 'https://www.reddit.com/r/cats/comments/d09fx6/stray_kitten_after_being_deflead_fed_and_bathed/'}, {'pontuacao': '11.7k', 'subreddit': '/r/cats', 'titulo thread': "Don't have many friends to share my cat with so here's how he sleeps", 'link para os comentarios': 'https://www.reddit.com/r/cats/comments/d0ewwv/im_so_lucky_to_have_her/'}, {'pontuacao': '9.1k', 'subreddit': '/r/cats', 'titulo thread': 'I’m so lucky to have her.', 'link para os comentarios': 'https://www.reddit.com/r/cats/comments/d0fovd/sweet_kitty_welcomes_returning_soldier/'}]
None
>:/NadaPraFazer dogs;worldnews

Top threads de /r/dogs:  Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)

Top threads de /r/worldnews:  [{'pontuacao': '33.1k', 'subreddit': '/

### Transferindo o bot para telegram

Primeiramente, precisamos criar um bot no servidor do telegram. Pra isso é necessário ter uma conta e logar no telegram. Na página do telegram, na caixa de busca, escreve-se "botfather". Clicando no primeiro perfil, que é um bot do telegram para criar bots, vai aparecer algumas opções, dentre elas a "/newbot". Seguindo os passos, é possível criar o bot inicial.

No meu caso, o nome escolhido foi 'RedScrappista', e o token de acesso é o:

989962435:AAHMclI0B6S7T5X88VI8ti6y4CkLN8UnJmg

Vamos então importar o módulo 'telepot' e criar o bot com o token dado:

In [3]:
import telepot
bot = telepot.Bot("989962435:AAHMclI0B6S7T5X88VI8ti6y4CkLN8UnJmg")

# Para pegar as mensagens que o bot recebeu (nesse caso eu enviei um 'oi'
# antes de rodar essa célula:
bot.getUpdates()

KeyboardInterrupt: 

In [85]:
# Para enviar mensagens, o primeiro argumento do método abaixo é o 'chat' 'id':
bot.sendMessage(937485481, 'olá lais')

{'message_id': 11,
 'from': {'id': 989962435,
  'is_bot': True,
  'first_name': 'RedScrappista',
  'username': 'RedScrappistaBot'},
 'chat': {'id': 937485481,
  'first_name': 'Laís',
  'last_name': 'Alves',
  'type': 'private'},
 'date': 1567816816,
 'text': 'olá lais'}

Caso vc tiver o telegram aberto, é possível gerar e verificar essas interações (desde que vc modifique o chat id) abrindo o bot no telegram através do link:

t.me/RedScrappistaBot

É possível também receber mensagens de uma forma mais fácil:

In [12]:
import telepot
bot = telepot.Bot("989962435:AAHMclI0B6S7T5X88VI8ti6y4CkLN8UnJmg")

def recebendoMsg(msg):
    print(msg['text'])

bot.message_loop(recebendoMsg)

while True:
    pass

olá, agora sao 8:05pm
e agora se passaram 20 segundos


KeyboardInterrupt: 

Sabendo como se envia e se recebe mensagens através dos métodos de bot do módulo "telepot", vamos então integrar o código do chatbot escrito anteriormente, com a interface do telegram:

In [2]:
import telepot
# from Chatbot import Chatbot
telegram = telepot.Bot("989962435:AAHMclI0B6S7T5X88VI8ti6y4CkLN8UnJmg")
bot = Chatbot("RedScrappista")

def recebendoMsg(msg):
    frase = bot.escuta(frase=msg['text'])
    resp = bot.pensa(frase)
    bot.fala(resp)
    chatID = msg['chat']['id']
    # Esse método abaixo é correspondente ao de cima
    # tipoMsg, tipoChat, chatID = telepot.glance(msg) 
    telegram.sendMessage(chatID, resp)

telegram.message_loop(recebendoMsg)

while True:
    pass
# O loop acima é um loop infinito para ser executado durante a conversa sem parar

KeyboardInterrupt: 

Dessa forma é possível enviar e receber mensagens para o bot através do telegram.

O único problema é que não consegui configurar o comando /NadaPraFazer, dentro do telegram. E quando envio o comando, aqui na minha máquina ele demora para ser processado.

(Gostaria de obter ajuda dos universitários nessa rs fiquei bem curiosa como fazer ^^)